In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import PIL
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')
import tensorflow_addons as tfa
from tensorflow.keras.metrics import top_k_categorical_accuracy, categorical_accuracy

In [2]:
root = "D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Complete Dataset"

In [4]:
train_dir = root+'/Hair Removed Images/train'
val_dir = root+'/Hair Removed Images/val'
test_dir = root+'/Hair Removed Images/test'
datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

# test_datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

val_data = datagen.flow_from_directory(val_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

test_data = datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=8,class_mode='categorical',shuffle=False)

Found 40253 images belonging to 7 classes.
Found 1000 images belonging to 7 classes.
Found 1006 images belonging to 7 classes.


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

num_labels = 7

base_model = ResNet50(include_top=False, input_shape=(128, 128, 3),pooling = 'avg', weights = 'imagenet')
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation = 'softmax',))


In [6]:
def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

lrReduction=ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.5,
                              patience=2,
                              verbose=1,
                              mode='auto',
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, verbose=1)

In [7]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',
                                                                        top_2_acc,
                                                                        top_3_acc,
                                                                        tf.metrics.AUC(curve='ROC'),
                                                                        tfa.metrics.MatthewsCorrelationCoefficient(num_classes=7)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 903       
Total params: 23,850,887
Trainable params: 23,797,767
Non-trainable params: 53,120
_________________________________________________________________


In [8]:
step_size_train=train_data.n//train_data.batch_size
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=step_size_train,
                   validation_data=val_data,
                    epochs=50, 
                    verbose = 1,
                    callbacks=[lrReduction,early_stopping])

Epoch 1/50
5031/5031 [==============================] - 850s 167ms/step - loss: 1.6395 - accuracy: 0.3566 - top_2_acc: 0.5440 - top_3_acc: 0.6937 - auc: 0.7462 - MatthewsCorrelationCoefficient: 0.2516 - val_loss: 1.7322 - val_accuracy: 0.2420 - val_top_2_acc: 0.4060 - val_top_3_acc: 0.5410 - val_auc: 0.7006 - val_MatthewsCorrelationCoefficient: 0.0899
Epoch 2/50
5031/5031 [==============================] - 821s 163ms/step - loss: 1.3594 - accuracy: 0.4804 - top_2_acc: 0.6722 - top_3_acc: 0.8058 - auc: 0.8383 - MatthewsCorrelationCoefficient: 0.3986 - val_loss: 4.6108 - val_accuracy: 0.1690 - val_top_2_acc: 0.4190 - val_top_3_acc: 0.5290 - val_auc: 0.5533 - val_MatthewsCorrelationCoefficient: 0.0911
Epoch 3/50
5031/5031 [==============================] - 853s 169ms/step - loss: 1.3005 - accuracy: 0.4966 - top_2_acc: 0.6883 - top_3_acc: 0.8189 - auc: 0.8517 - MatthewsCorrelationCoefficient: 0.4172 - val_loss: 1.3164 - val_accuracy: 0.5510 - val_top_2_acc: 0.6890 - val_top_3_acc: 0.7950 -

In [9]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(train_data)

5032/5032 [==============================] - 263s 52ms/step - loss: 0.4795 - accuracy: 0.9833 - top_2_acc: 0.9973 - top_3_acc: 0.9998 - auc: 0.9986 - MatthewsCorrelationCoefficient: 0.9805


In [10]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(val_data)

125/125 [==============================] - 7s 52ms/step - loss: 1.0556 - accuracy: 0.7710 - top_2_acc: 0.9130 - top_3_acc: 0.9790 - auc: 0.9471 - MatthewsCorrelationCoefficient: 0.5434


In [11]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(test_data)

126/126 [==============================] - 12s 98ms/step - loss: 1.0636 - accuracy: 0.7535 - top_2_acc: 0.9105 - top_3_acc: 0.9702 - auc: 0.9435 - MatthewsCorrelationCoefficient: 0.5234


In [12]:
from sklearn import metrics

true_test_labels=test_data.labels

true_test_labels=np.array(true_test_labels)

np.unique(true_test_labels,return_counts=True)

pred_test_labels=model.predict(test_data)

pred_test_labels=np.argmax(pred_test_labels,axis=1)

metrics.accuracy_score(true_test_labels,pred_test_labels)

report = metrics.classification_report(true_test_labels,pred_test_labels)
print(report)

              precision    recall  f1-score   support

           0       0.47      0.44      0.45        34
           1       0.52      0.62      0.57        52
           2       0.47      0.45      0.46       111
           3       0.67      0.50      0.57        12
           4       0.54      0.43      0.48       112
           5       0.86      0.89      0.87       670
           6       0.72      0.87      0.79        15

    accuracy                           0.75      1006
   macro avg       0.61      0.60      0.60      1006
weighted avg       0.75      0.75      0.75      1006



In [13]:
# model.save("D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Saved Models\Resnet_50.h5")